<a href="https://colab.research.google.com/github/valsson-group/UNT-Chem5210-Spring2025/blob/main/Python-JupyterNotebooks/TDDFT_ExcitedStates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Excited State from TD-DFT

Simple Juypter notebook that performs TD-DFT (Time-dependent density-functional theory) using pyscf.

Part of the code take from and inspired by https://www.andersle.no/posts/2022/mo/mo.html

## Setup python environment and imports

In [ ]:
%%capture
!pip install numpy
!pip install scipy
!pip install Cython
!pip install pandas
!pip install sklearn
!pip install statsmodels
!pip install matplotlib
!pip install rdkit
!pip install py3Dmol
!pip install ipywidgets
!pip install sphinx
!pip install nbsphinx
!pip install pydata-sphinx-theme
!pip install lxml
!pip install fortecubeview
!pip install pythreejs
!pip install scikit-image
!pip install pyscf
!pip install shap
!pip install catboost
!pip install graphviz

In [ ]:
import pathlib

# RDKit imports:
from rdkit import Chem
from rdkit.Chem import (
    AllChem,
    rdCoordGen,
)
from rdkit.Chem.Draw import IPythonConsole

IPythonConsole.ipython_useSVG = True  # Use higher quality images for molecules

# For visualization of molecules and orbitals:
import py3Dmol
import fortecubeview

# pyscf imports:
from pyscf import gto, scf, dft, tddft, tools

from pyscf.data.nist import HARTREE2EV
from pyscf.data.nist import HARTREE2WAVENUMBER

# For plotting
import matplotlib
from matplotlib import pyplot as plt

%matplotlib inline

# For numerics:
import numpy as np

from google.colab import output
output.enable_custom_widget_manager()

## Function Definitions

In [ ]:
def get_xyz(molecule, optimize=False):
    """Get xyz-coordinates for the molecule"""
    mol = Chem.Mol(molecule)
    mol = AllChem.AddHs(mol, addCoords=True)
    AllChem.EmbedMolecule(mol)
    if optimize:  # Optimize the molecules with the MM force field:
        AllChem.MMFFOptimizeMolecule(mol)
    xyz = []
    for lines in Chem.MolToXYZBlock(mol).split("\n")[2:]:
        strip = lines.strip()
        if strip:
            xyz.append(strip)
    xyz = "\n".join(xyz)
    return mol, xyz

In [ ]:
def run_calculation_tddft(xyz, functional="b3lyp", basis="sto-3g", nstates=5,symmetry=False):
    """Calculate the energy (+ additional things like MO coefficients) with pyscf."""
    mol = gto.M(
        atom=xyz,
        basis=basis,
        unit="ANG",
        symmetry=symmetry,
    )
    mol.build()
    mf = dft.RKS(mol)
    mf.xc = functional
    mf.kernel()
    td = tddft.TDDFT(mf)
    td.nstates = nstates
    td.kernel()
    return mf, mol, td

In [ ]:
def write_cube_files(
    max_homo_lumo=5, prefix="", dirname=".", margin=5, write_all_orbitals=False
):
    """Write cube files for the given coefficients."""
    path = pathlib.Path(dirname)
    path.mkdir(parents=True, exist_ok=True)

    # find index of HOMO and LUMO
    lumo = float("inf")
    lumo_idx = None
    homo = -float("inf")
    homo_idx = None
    for i, (energy, occ) in enumerate(zip(mf.mo_energy, mf.mo_occ)):
        if occ > 0 and energy > homo:
            homo = energy
            homo_idx = i
        if occ == 0 and energy < lumo:
            lumo = energy
            lumo_idx = i

    if(write_all_orbitals):
      for i in range(mf.mo_coeff.shape[1]):
        outfile = f"{prefix}Orbital-{i:02d}.cube"
        outfile = path / outfile
        print(f"Writing {outfile}")
        tools.cubegen.orbital(mol, outfile, mf.mo_coeff[:, i], margin=margin)
    else:
      print(f"HOMO (index): {homo_idx+1}")
      print(f"LUMO (index): {lumo_idx+1}")
      print("")
      outfile = f"{prefix}HOMO.cube"
      outfile = path / outfile
      print(f"Writing {outfile}")
      tools.cubegen.orbital(mol, outfile, mf.mo_coeff[:, homo_idx], margin=margin)

      outfile = f"{prefix}LUMO.cube"
      outfile = path / outfile
      print(f"Writing {outfile}")
      tools.cubegen.orbital(mol, outfile, mf.mo_coeff[:, lumo_idx], margin=margin)

      for i in range(1,max_homo_lumo+1):
        outfile = f"{prefix}HOMO_minus-{i:02d}.cube"
        outfile = path / outfile
        print(f"Writing {outfile}")
        tools.cubegen.orbital(mol, outfile, mf.mo_coeff[:, homo_idx-i], margin=margin)

        outfile = f"{prefix}LUMO_plus-{i:02d}.cube"
        outfile = path / outfile
        print(f"Writing {outfile}")
        tools.cubegen.orbital(mol, outfile, mf.mo_coeff[:, lumo_idx+i], margin=margin)

In [ ]:
def GaussianPeak(x,mean,sigma):
  return np.exp(-(x-mean)**2/(2*sigma**2))

def plot_spectrum_eV(exc,osc,width=0.2,x_range=None):
  if x_range is None:
    x_min = round(exc[0]-4*width)
    x_max = round(exc[-1]+4*width+0.5)
  else:
    x_min = x_range[0]
    x_max = x_range[1]
  x=np.linspace(x_min,x_max,1000)
  y=np.zeros(x.size)
  for i, e in enumerate(exc):
    if not np.isnan(osc[i]):
      y+=osc[i]*GaussianPeak(x,e,width)

  plt.plot(x,y)
  plt.xlabel("Excitation [eV]")
  plt.ylabel("Intensity")
  plt.xlim([x_min,x_max])
  for i, e in enumerate(exc):
    if not np.isnan(osc[i]):
      plt.stem(e,osc[i],linefmt="--")
  plt.show()

def plot_spectrum_nm(exc,osc,width=10,x_range=None):
  if x_range is None:
    x_min = round(exc[-1]-4*width+0.5)
    x_max = round(exc[0]+4*width)
  else:
    x_min = x_range[0]
    x_max = x_range[1]
  x=np.linspace(x_min,x_max,1000)
  y=np.zeros(x.size)
  for i, e in enumerate(exc):
    if not np.isnan(osc[i]):
      y+=osc[i]*GaussianPeak(x,e,width)

  plt.plot(x,y)
  plt.xlabel("Wavelength [nm]")
  plt.ylabel("Intensity")
  plt.xlim([x_min,x_max])
  for i, e in enumerate(exc):
    if not np.isnan(osc[i]):
      plt.stem(e,osc[i],linefmt="--")
  plt.show()


## Definition of Molecule from SMILES String

**Here you should define the smiles string for the molecule you want to consider**




In [ ]:
molecule_smiles = "[O-][O+]=O" #@param {type:"string"}

molecule_name = "Mol"
molecule = Chem.MolFromSmiles(molecule_smiles)  # Generate the molecule from smiles
molecule

## Setup of Molecule

In [ ]:
molecule3d, xyz = get_xyz(molecule, optimize=True)
print(xyz)

In [ ]:
view = py3Dmol.view(
    data=Chem.MolToMolBlock(molecule3d),
    style={"stick": {}, "sphere": {"scale": 0.3}},
    width=300,
    height=300,
  )
view.zoomTo()

## DFT and TD-DFT Calculations

Here we perform the DFT and TD-DFT calculations. You will need to select the DFT functional and the basis set to be used from the calculation from the drop down lists. You can also select the number of excited states to be calculated for in the TD-DFT calculations, and if you wish to use symmetry in the calculations.

In [ ]:
DFT_Functional = "BLYP" #@param {type:"string"} ["BLYP", "B3LYP", "PBE0", "wB97XD"]
BasisSet = "6-31G(d)" #@param {type:"string"} ["sto-3g", "cc-pVDZ","6-31G(d)","def2-SVP"]
NumberOfStates = 10 # @param {type:"integer"}
UseSymmetry = True # @param {type:"boolean"}

print("Running DFT calculations")
print("- Functional: {:s}".format(DFT_Functional))
print("- Basis set: {:s}".format(BasisSet))


mf, mol, td = run_calculation_tddft(xyz, functional=DFT_Functional, basis=BasisSet ,nstates=NumberOfStates,symmetry=UseSymmetry)

## Analysis of Results

### DFT Results



In [ ]:
mf.analyze(verbose=3)

### TD-DFT Results

In [ ]:
td.analyze(verbose=4)

## Plot Spectrum

Here we plot the spectra by taking each excitation as a Gaussian peak with a finite width that represent broading effects. Each Gaussian peak is weighted with the oscillator strength.

In [ ]:
exc_eV=td.e*HARTREE2EV
osc=td.oscillator_strength()
exc_nm=1e7/(td.e*HARTREE2WAVENUMBER)

# print(exc_eV)
# print(exc_nm)

# Peak width in eV
peak_width_eV=0.2
peak_width_nm=10

plot_spectrum_eV(exc_eV,osc,width=peak_width_eV)
plot_spectrum_nm(exc_nm,osc,width=peak_width_nm)


## Setup Orbitals

In [ ]:
!rm -rf cube_files
write_cube_files(
   dirname="cube_files",
   write_all_orbitals=False
)

## Visualize Orbitals

In [ ]:
fortecubeview.plot(path="./cube_files/", width=600, height=300, colorscheme='national', sumlevel=0.7)